In [0]:
""" 
Descripcion:
     Lectura de archivo 070 (respuesta del archivo 69) y no fueron desmarcados.
Subetapa: 
    25 - Matriz de Convivnecia
Trámite:
    COMUN - 
Tablas input:
   
Tablas output:
    PROCESOS.THAFOGRAL_MAR_DES_ITGY
    PROCESOS.TTAFOGRAL_RESPUESTA_ITGY
    
Tablas Delta:
    TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_{params.sr_folio}
    TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_ENC_{params.sr_folio}

Archivos SQL:
    COMUN_MCV_150_DB_20_DEL_PROCESOS_THAFOGRAL_MAR_DES_ITGY.sql
    COMUN_MCV_150_DB_09_DEL_TTAFOGRAL_RESPUESTA_ITGY_AUX.sql
    COMUN_MCV_DESM_050_EXTRAE_INFO_RESPUESTA_70_NO_DESMARCADOS.sql
    """

30-01-2026 - Se corrige extension del archivo generado, de agega delay para evitar error de inconsistencia, mueve archivo (FOLIO_DESMARCA_INTEGRITY.**csv**) generado a la ruta /INTEGRITY/INF/
2026-02-09 - Se modifica SQL para realizar correctamente la carga de FHD_FEH_MARCA y FHD_FEH_DESMARCA

In [0]:
%run "./startup"

In [0]:
# Crear la instancia con los parámetros esperados
params = WidgetParams({
    "sr_proceso": str,
    "sr_subproceso": str,
    "sr_subetapa": str,
    "sr_folio": str,
    "sr_usuario": str,
    "sr_path_arch": str,
    "sr_conv_ingty": str,
    "sr_tipo_ejecucion":str,
    #valores obligatorios
    "sr_etapa": str,
    "sr_instancia_proceso": str,    
    "sr_id_snapshot": str,
})
# Validar widgets
params.validate()

In [0]:
# 🚀 **Ejemplo de Uso**
conf = ConfManager()
#Archivos SQL
query = QueryManager()
#Conexion a base de datos
db = DBXConnectionManager()

In [0]:
# Define la ruta del archivo 70 
# #p_PATH_INTEGRITY_ARC##p_SR_NOM_RESPUESTA##p_SR_FOLIO##p_CLV_SUFIJO_3#.DAT
# path_arch_70 =  f"{params.sr_path_arch}"+ "ArchivosOK" + f"/PPA_RCAT070_{params.sr_folio}" + "_3.DAT"
# path_arch_70 =  f"{params.sr_path_arch}"+ f"/PPA_RCAT076_{params.sr_folio}" + "_3.DAT"
# El parametro de la ruta completa se recibe de servicios 2026-01-23 CRB
path_arch_70 =  f"{params.sr_path_arch}"

In [0]:
#
# Este codigo ya no se usa porque se recibe la ruta completa por parametro CRB 2026-01-23
#
ruta = SETTINGS.GENERAL.EXTERNAL_LOCATION + f"{params.sr_path_arch}" 
# Listar los archivos y carpetas en la ruta especificada
# archivos = dbutils.fs.ls(ruta)
# Inicializar la variable
# nombre_encontrado = None
# Buscar si existe "ArchivosOK" o "archivosOK" y asignar el valor encontrado
# for f in archivos:
#    nombre = f.name.rstrip('/')
#    if nombre in ["ArchivosOK", "archivosOK"]:
#        nombre_encontrado = nombre
#        break
# if nombre_encontrado:
#    print(f"Existe '{nombre_encontrado}' en la ruta.")
# else:
#    print("No existe 'ArchivosOK' ni 'archivosOK' en la ruta.")

# dbutils.fs.ls('abfss://nci-repository@datalakedev1udbvf.dfs.core.windows.net/INTEGRITY/OK/')

print(ruta)
print(path_arch_70)
print(SETTINGS.GENERAL.EXTERNAL_LOCATION)


In [0]:
#from pyspark.sql.functions import monotonically_increasing_id

def load_file(storage_location, path_arch_70):    
    # Leer el archivo de texto
    df = spark.read.text(storage_location + path_arch_70)
    
    # Renombrar la columna "value" a "FTC_LINEA"
    df = df.withColumnRenamed("value", "FTC_LINEA")
    
    # Agregar un número de línea único
    indexed_df = df.withColumn("FTN_NO_LINEA", monotonically_increasing_id() + 1)
    
    return indexed_df


In [0]:
# dfLoaded = load_file(SETTINGS.GENERAL.EXTERNAL_LOCATION, f"{params.sr_path_arch}"+ "/archivosOk" + f"/PPA_RCAT065_{params.sr_folio}" + "_2.DAT")
dfLoaded = load_file(SETTINGS.GENERAL.EXTERNAL_LOCATION, path_arch_70)
if conf.debug:
    display(dfLoaded)

In [0]:
delta_001 = f"TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_{params.sr_folio}"
db.write_delta(delta_001, dfLoaded, "overwrite")

if conf.debug:
    display(db.read_delta(f"TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_{params.sr_folio}"))


In [0]:
# Query para los datos para TTAFOGRAL_RESPUESTA_ITGY 
statement_RI_DET = query.get_statement(
    "COMUN_MCV_DESM_050_EXTRAE_INFO_RESPUESTA_70_DET.sql",
    sr_folio=params.sr_folio,
    DELTA_TABLA_NAME1 = f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}.TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_{params.sr_folio}",
 #   FECHA_PROCESO = fecha_actual
 )

db.write_delta(f"TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_DET_{params.sr_folio}", db.sql_delta(statement_RI_DET), "overwrite")

if conf.debug:
    display(db.read_delta(f"TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_DET_{params.sr_folio}"))

In [0]:
%sql
-- DESCRIBE TABLE dbx_mit_dev_1udbvf_workspace.default.TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_DET_202602061616274420;

In [0]:

#  -------------------------------------------------------------------------------------
# Eliminamos registros en THAFOGRAL_MAR_DES_ITGY 
# Datastage elimina y despues inserta 
# COMUN_MCV_150_DB_20_DEL_PROCESOS_THAFOGRAL_MAR_DES_ITGY.sql
#  PROCESOS.THAFOGRAL_MAR_DES_ITGY
# ELIMINA REGISTROS DEL FOLIO EN respuesta THAFOGRAL_MAR_DES_ITGY
statement_del_mar_des = query.get_statement(
    "COMUN_MCV_150_DB_20_DEL_PROCESOS_THAFOGRAL_MAR_DES_ITGY.sql",
    SR_FOLIO=params.sr_folio,
    hints="/*+ PARALLEL(8) */",
)
execution = db.execute_oci_dml(
    statement=statement_del_mar_des, async_mode=False
)

#INSERTA DIRECTAMENTE EN LA TABLA (ASI FUNCIONA EN DATASTAGE)

table_name = "PROCESOS.THAFOGRAL_MAR_DES_ITGY"

db.write_data(db.read_delta(f"TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_DET_{params.sr_folio}"), table_name, "default", "append")

In [0]:
# Query para los datos para TTAFOGRAL_RESPUESTA_ITGY 
statement_RI_ENC = query.get_statement(
    "COMUN_MCV_DESM_050_EXTRAE_INFO_RESPUESTA_70_ENC.sql",
    SR_FOLIO=params.sr_folio,
    SR_PROCESO=params.sr_proceso,
    SR_SUBPROCESO=params.sr_subproceso,
    DELTA_TABLA_NAME1 = f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}.TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_{params.sr_folio}",
 #   FECHA_PROCESO = fecha_actual
 )

In [0]:
db.write_delta(f"TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_ENC_{params.sr_folio}", db.sql_delta(statement_RI_ENC), "overwrite")

if conf.debug:
    display(db.read_delta(f"TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_ENC_{params.sr_folio}"))

In [0]:

# ELIMINA REGISTROS DEL FOLIO EN respuesta integrity aux
statement = query.get_statement(
    "COMUN_MCV_150_DB_09_DEL_TTAFOGRAL_RESPUESTA_ITGY_AUX.sql",
    SR_FOLIO=params.sr_folio,
    hints="/*+ PARALLEL(8) */",
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

#INSERTA EN RESPUESTA INTEGRITY AUX

table_name = "CIERREN_DATAUX.TTAFOGRAL_RESPUESTA_ITGY_AUX"

db.write_data(db.read_delta(f"TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_ENC_{params.sr_folio}"), table_name, "default", "append")

# REALIZA MERGE TTAFOGRAL_RESPUESTA_ITGY_AUX Y TTAFOGRAL_RESPUESTA_ITGY
statement = query.get_statement(
    "COMUN_MCV_150_DB_10_MERGE_TTAFOGRAL_RESPUESTA_ITGY_AUX_UPD.sql",
    SR_FOLIO=params.sr_folio,
    hints="/*+ PARALLEL(8) */",
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)


In [0]:
# COMUN_MCV_DESM_050_EXTRAE_INFO_RESPUESTA_70_NO_DESMARCADOS.sql
statement_RI_NODESM = query.get_statement(
    "COMUN_MCV_DESM_050_EXTRAE_INFO_RESPUESTA_70_NO_DESMARCADOS.sql",
    SR_FOLIO=params.sr_folio,
    SR_PROCESO=params.sr_proceso,
    SR_SUBPROCESO=params.sr_subproceso,
    DELTA_TABLA_NAME1 = f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}.TEMP_DELTA_COMUN_MCV_DELTA_DESM_RESP_ITGY_ARCH_70_{params.sr_folio}", 
 )

# sql_delta --> para consultar tablas delta con un query
# read_delta --> para hacerle un select * from a una tabla delta sin necesidad de usar query

df_from_delta=db.sql_delta(statement_RI_NODESM)

if conf.debug:
    display(df_from_delta)



In [0]:
# df_from_delta
if len(df_from_delta.take(1)) == 1:
    # Define la ruta final donde quieres guardar el archivo .dat
    final_file_path = SETTINGS.GENERAL.EXTERNAL_LOCATION + '/INTEGRITY/INF/'+ f"/ANT_{params.sr_folio}" + "_DESMARCA_INTEGRITY.CSV"
    definitive_file_path = SETTINGS.GENERAL.EXTERNAL_LOCATION + '/INTEGRITY/INF/'+ f"/{params.sr_folio}" + "_DESMARCA_INTEGRITY.csv"

# 1. Guarda el DataFrame como texto en una carpeta temporal
    temp_path = final_file_path.replace('.CSV', '_temp')  # Cambia la extensión para evitar conflictos
    #  asi solo genera formato unix LF al final
    #  df_final.coalesce(1).write.mode("overwrite").text(temp_path)
    #  genera en formato windows con CRLF
    # df_from_delta.coalesce(1).write.mode("overwrite").option("lineSep", "\r\n").text(temp_path)

    # Escribe un CSV (posible múltiples part- files; usamos coalesce(1) si queremos un único archivo)
    (df_from_delta
        .coalesce(1) \
        .write \
        .mode("overwrite") \
        .option("header", "true") \
        .option("sep", ",") \
        .option("lineSep", "\r\n") \
        .option("encoding", "UTF-8") \
        .csv(temp_path))
    
    # Wait for Azure storage consistency
    import time
    time.sleep(10)
    
# 2. Mueve y renombra el archivo que se genera
# La ruta del archivo generado señalado por el sistema será algo así como temp_path/part-00000-...
    import glob

# Busca el archivo en la carpeta temporal
    import os

    # Listar archivos generados en la carpeta temporal
    temp_files = dbutils.fs.ls(temp_path)

    # Verificamos si hay archivos y movemos el primero encontrado
    if temp_files:
        for file_info in temp_files:
            # Asegúrate de que solo mueves el archivo correcto
            if file_info.name.endswith(".csv"):
                dbutils.fs.mv(file_info.path, final_file_path)
                break

    # 3. Elimina la carpeta temporal que se creó
    dbutils.fs.rm(temp_path, recurse=True)
 
    # 4. Renombra el archivo final listo para que lo tome el servicio de busqueda
    dbutils.fs.mv(final_file_path, definitive_file_path)
    print(f"Archivo generado en: {definitive_file_path}")
else:
    print("No hay datos para generar el archivo")

In [0]:
CleanUpManager.cleanup_notebook(locals())

In [0]:
Notify.send_notification("DESMARCA", params)